# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [34]:
import pandas as pd
import numpy as np

In [35]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,0.694455,0.181015,-1.052072
c,-0.132827,-2.009981,1.201623
e,-0.820473,0.741852,-0.166862
f,-0.723541,0.204186,1.068059
h,0.407758,-1.496639,0.099934


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [36]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,0.694455,0.181015,-1.052072,bar,True
c,-0.132827,-2.009981,1.201623,bar,False
e,-0.820473,0.741852,-0.166862,bar,False
f,-0.723541,0.204186,1.068059,bar,False
h,0.407758,-1.496639,0.099934,bar,True


Adding new rows is also simple. Below we include three extra empty rows:

In [37]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,0.694455,0.181015,-1.052072,bar,True
b,NaN,NaN,NaN,NaN,NaN
c,-0.132827,-2.009981,1.201623,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,-0.820473,0.741852,-0.166862,bar,False
f,-0.723541,0.204186,1.068059,bar,False
g,NaN,NaN,NaN,NaN,NaN
h,0.407758,-1.496639,0.099934,bar,True


# Your Turn Here

Do you still remember how to index a row/column?

In [38]:
#### index row 'c' below
df2.loc['c']


one     -0.132827
two      -2.00998
three     1.20162
four          bar
five        False
Name: c, dtype: object

In [39]:
#### index column 'two' below
df.two

a    0.181015
c   -2.009981
e    0.741852
f    0.204186
h   -1.496639
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [40]:
pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [41]:
pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [42]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/series.py:935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_with(key, value)
/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,one,two
a,NaN,0.181015
c,NaN,-2.009981
e,NaN,0.741852
f,-0.723541,0.204186
h,0.407758,-1.496639


In [43]:
b = df[['one','two','three']]
b

,one,two,three
a,0.694455,0.181015,-1.052072
c,-0.132827,-2.009981,1.201623
e,-0.820473,0.741852,-0.166862
f,-0.723541,0.204186,1.068059
h,0.407758,-1.496639,0.099934


In [44]:
a + b

,one,three,two
a,NaN,NaN,0.362030
c,NaN,NaN,-4.019962
e,NaN,NaN,1.483704
f,-1.447082,NaN,0.408371
h,0.815516,NaN,-2.993278


In [45]:
b + a

,one,three,two
a,NaN,NaN,0.362030
c,NaN,NaN,-4.019962
e,NaN,NaN,1.483704
f,-1.447082,NaN,0.408371
h,0.815516,NaN,-2.993278


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [46]:
a['one'].dropna()

f   -0.723541
h    0.407758
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [47]:
#### This statement drop any column with NaN values
a.dropna(axis=1)

,two
a,0.181015
c,-2.009981
e,0.741852
f,0.204186
h,-1.496639


In [48]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

,one,two
a,NaN,0.362030
c,NaN,-4.019962
e,NaN,1.483704
f,-1.447082,0.408371
h,0.815516,-2.993278


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [49]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

,one,two
a,NaN,0.362030
c,NaN,-4.019962
e,NaN,1.483704
f,-1.447082,0.408371
h,0.815516,-2.993278


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [50]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [51]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [52]:
# forward-fill
my_series.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [53]:
my_series.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [54]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,0.115035,0.555689,NaN
b,-0.069245,NaN,1.231796
c,-2.245973,NaN,-0.685010
d,0.897676,NaN,0.086039
e,NaN,NaN,NaN


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [57]:
#### insert your code here

rand_df.one.fillna(1)
rand_df.two.fillna(method='ffill')
rand_df.three.fillna(method='bfill')
rand_df

,one,two,three
a,0.115035,0.555689,NaN
b,-0.069245,NaN,1.231796
c,-2.245973,NaN,-0.685010
d,0.897676,NaN,0.086039
e,NaN,NaN,NaN


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [58]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,NaN,NaN,NaN
b,0.237440,-0.369211,0.746439
c,-0.890869,NaN,NaN
d,0.608496,NaN,NaN
e,NaN,1.065797,NaN


In [59]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

True

In [60]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

A   -0.014978
B    0.348293
C    0.746439
dtype: float64

In [61]:
df_filled = my_df.fillna(my_df.mean())
df_filled

,A,B,C
a,-0.014978,0.348293,0.746439
b,0.237440,-0.369211,0.746439
c,-0.890869,0.348293,0.746439
d,0.608496,0.348293,0.746439
e,-0.014978,1.065797,0.746439


In [62]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.